In [3]:
import sys
sys.path.append("../")

from document_search.ocr import DocumentReader, EntityProcessor


In [4]:
documents = [
    "../data/SP-51.13330.2011.pdf",
    "../data/SP-60.13330.2020.pdf"
]


In [5]:
doc_reader = DocumentReader()

processed_docs = []
for document_path in documents:
    processed_doc, errors = doc_reader.process_pdf(document_path)
    print(document_path, errors)
    processed_doc.entities = EntityProcessor.merge_text_entities(processed_doc.entities)
    processed_doc.entities = EntityProcessor.filter_short_entities(processed_doc.entities)
    processed_docs.append(processed_doc)


../data/SP-51.13330.2011.pdf []
../data/SP-60.13330.2020.pdf []


In [6]:
from document_search.storages import DocumentStorageE5
from document_search.search import TextEntityEmbedderE5

embedder = TextEntityEmbedderE5()
storage = DocumentStorageE5(embedder)


/home/atom/igor/hacks/document-search/notebooks/../document_search/search/embedders.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedder = HuggingFaceEmbeddings(
/home/atom/anaconda3/envs/doc-search2/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
for doc in processed_docs:
    storage.add_document(doc, pbar=True)


100%|██████████| 53/53 [00:51<00:00,  1.03it/s]
126it [00:57,  2.20it/s]                         


In [8]:
query = "здания, в которых на фасаде, обращенном в сторону внешнего источника шума, установлены шумозащитные окна"

storage.get_relevant_entities(query, 5)


[TextDocEntity(position=EntityPosition(document_name='SP-51.13330.2011', page_number=6), text='здания, в которых на фасаде, обращенном в сторону внешнего источника шума, установлены шумозащитные окна, снабженные специальными вентиляционными устройствами с глушителями шума;'),
 TextDocEntity(position=EntityPosition(document_name='SP-51.13330.2011', page_number=47), text='конструктивных  мер,  предусматривающих  строительство  придорожных  экранов,  установку шумозащитных окон в зданиях, расположенных в зонах неблагоприятного шумового воздействия;'),
 TextDocEntity(position=EntityPosition(document_name='SP-51.13330.2011', page_number=6), text='3.23 шумозащитные здания: Жилые здания, к которым относятся: здания  со  специальным  архитектурно-планировочным  решением,  предусматривающим ориентацию в сторону источника шума (магистрали) подсобных помещений квартир (кухни, ванные комнаты, санузлы), внеквартирных коммуникаций (лестнично-лифтовые узлы, коридоры), а также не более одной комнаты в

In [9]:
query = "какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?"

storage.get_relevant_entities(query, 5)


[TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020', page_number=66), text='10.6  Через  помещение  для  вентиляционного  оборудования  не  допускается  прокладывать трубопроводы: а) с легковоспламеняющимися и горючими жидкостями и газами;'),
 TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020', page_number=77), text='Не  допускается  прокладка  магистральных  и  разводящих  трубопроводов  систем  отопления  и внутреннего теплоснабжения через помещения жилых квартир, а также установка в них арматуры и спускных устройств общедомовых систем.'),
 TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020', page_number=50), text='7.10.14  Оборудование  вытяжных  систем  производственных,  административно-бытовых  и общественных  зданий,  удаляющих  воздух  с  резким  или  неприятным  запахом  (из  общественных уборных,  курительных  комнат  и  др.),  не  допускается  размещать  в  общем  помещении  для вентиляционного оборудования вместе с 